In [44]:
#expand cell width to 100%
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [45]:
#import pandas
import pandas as pd
#import pyplot from matplotlib
import matplotlib.pyplot as plt
#import numpy
import numpy as np
#import regex
import re

In [46]:
#create a list containing the formula of elements that are of interest
elementsOfInterest=['Al','Ca','Fe','Mg','O','Si','Ti','K','Th','U']

In [47]:
#set the number of lines to be retrieved per element per dataset
noOfLinesPerElementPerDataset=3

In [48]:
#compute the eneergy values for each band
gain=17.8 #keV/channel
energyBands=np.arange(0,512,1)*gain
#get the index of the last relavent band
finalRelaventBandIndex=np.argmin(np.abs(energyBands-8000))
#compute the index of the first relavent band
firstRelaventBandIndex=finalRelaventBandIndex-421+1
#get the energies of the relavent bands
relaventEnergyBands=energyBands[firstRelaventBandIndex:finalRelaventBandIndex+1]

In [49]:
#set the address of the Capture IAEA file
iaeaFileAddress="C:/ML4Sci/NASA-gamma/nasagamma/data/Capture_IAEA.csv"

In [50]:
#read the csv file as a dataframe
captureIAEADataframe=pd.read_csv(iaeaFileAddress)

In [51]:
#show the read dataframe
captureIAEADataframe

,Isotope,Energy (keV),Sigma (mb),DSigma (mb)
0,1H,2223.248,3.330000e+02,7.000000e-01
1,2H,6250.243,5.190000e-01,7.000000e-03
2,3He,20520.460,4.200000e-08,1.200000e-08
3,6Li,7245.910,2.470000e+00,1.400000e-01
4,6Li,477.595,1.530000e+00,8.000000e-02
...,...,...,...,...
31611,238U,3743.920,1.400000e-01,6.000000e-02
31612,238U,4071.750,1.400000e-01,4.000000e-02
31613,238U,3913.100,8.000000e-02,4.000000e-02
31614,238U,4806.380,7.000000e-02,7.000000e-02


In [52]:
#retain rows for only the elements of interest
captureIAEADataframe=captureIAEADataframe[captureIAEADataframe['Isotope'].str.endswith(tuple(elementsOfInterest))]

In [53]:
#remove any rows that are outside the relavent energy range
captureIAEADataframe=captureIAEADataframe[captureIAEADataframe["Energy (keV)"].between(relaventEnergyBands[0],relaventEnergyBands[-1])]

In [54]:
#create a temporary variable to be used to create the dataframe containing only the sampled lines
temp=[]
#iterate through the elements of interest
for currentElement in elementsOfInterest:
    #get the rows corresponding to the current element
    currentElementFrame=captureIAEADataframe[captureIAEADataframe['Isotope'].str.endswith(tuple(currentElement))]
    #sort the dataframe in increasing order of cross-section
    currentElementFrame=currentElementFrame.sort_values('Sigma (mb)')
    #remove any values less than a certain Sigma threshold
    #currentElementFrame=currentElementFrame[currentElementFrame['Sigma (mb)']>=10]
    #sample the last few rows (number specified in the beginning)
    currentElementFrame=currentElementFrame.iloc[-1*noOfLinesPerElementPerDataset:,:]
    #add a column with the name of the current element
    currentElementFrame['Element']=currentElement
    #save the extracted rows
    if currentElement=='Al':
        temp=currentElementFrame
    else:
        temp=pd.concat([temp,currentElementFrame])
#save temp as the Bagddad dataframe
captureIAEADataframe=temp

In [55]:
#add a column Type and set it to Capture_IAEA
captureIAEADataframe["Type"]="Capture_IAEA"

In [56]:
#show the processed dataframe
captureIAEADataframe

,Isotope,Energy (keV),Sigma (mb),DSigma (mb),Element,Type
860,27Al,4259.534,15.300,0.300,Al,Capture_IAEA
859,27Al,3033.896,17.900,0.300,Al,Capture_IAEA
858,27Al,7724.027,49.300,1.500,Al,Capture_IAEA
2609,40Ca,4418.520,70.800,1.800,Ca,Capture_IAEA
2606,40Ca,6419.590,176.000,5.000,Ca,Capture_IAEA
2604,40Ca,1942.670,352.000,7.000,Ca,Capture_IAEA
5848,56Fe,6018.532,227.000,5.000,Fe,Capture_IAEA
5846,56Fe,7645.545,549.000,11.000,Fe,Capture_IAEA
5845,56Fe,7631.136,653.000,13.000,Fe,Capture_IAEA
577,24Mg,2828.172,24.000,0.800,Mg,Capture_IAEA


In [57]:
#set the address for the Inelastic_Baghdad file
InelasticBaghdadFileAddress="C:/ML4Sci/NASA-gamma/nasagamma/data/Inelastic_Baghdad.csv"

In [58]:
#read the .csv file into a dataframe
inelasticBaghdadDataframe=pd.read_csv(InelasticBaghdadFileAddress)

In [59]:
#show the read dataframe
inelasticBaghdadDataframe

,Isotope,Energy (keV),XS (mb),Compound
0,00Li,478.40,248.040,7Li
1,00B,477.70,61.308,7Li
2,00B,2124.00,5.850,11B
3,00B,718.18,2.223,10B
4,00B,4442.20,2.223,11B
...,...,...,...,...
7371,00U,1113.60,2.986,238U
7372,00U,1263.60,2.714,238U
7373,00U,1103.60,2.172,238U
7374,00U,1524.50,2.172,238U


In [60]:
#retain rows for only the elements of interest
inelasticBaghdadDataframe=inelasticBaghdadDataframe[inelasticBaghdadDataframe['Isotope'].str.endswith(tuple(elementsOfInterest))]

In [61]:
#remove any rows that are outside the relavent energy range
inelasticBaghdadDataframe=inelasticBaghdadDataframe[inelasticBaghdadDataframe["Energy (keV)"].between(relaventEnergyBands[0],relaventEnergyBands[-1])]

In [62]:
#create a temporary variable to be used to create the dataframe containing only the sampled lines
temp=[]
#iterate through the elements of interest
for currentElement in elementsOfInterest:
    #get the rows corresponding to the current element
    currentElementFrame=inelasticBaghdadDataframe[inelasticBaghdadDataframe['Isotope'].str.endswith(tuple(currentElement))]
    #sort the dataframe in increasing order of cross-section
    currentElementFrame=currentElementFrame.sort_values('XS (mb)')
    #remove any values less than a certain Sigma threshold
    #currentElementFrame=currentElementFrame[currentElementFrame['XS (mb)']>=10]
    #sample the last few rows (number specified in the beginning)
    currentElementFrame=currentElementFrame.iloc[-1*noOfLinesPerElementPerDataset:,:]
    #add a column with the name of the current element
    currentElementFrame['Element']=currentElement
    #save the extracted rows
    if currentElement=='Al':
        temp=currentElementFrame
    else:
        temp=pd.concat([temp,currentElementFrame])
#save temp as the Bagddad dataframe
inelasticBaghdadDataframe=temp

In [63]:
#add a column Type and set it to Inelastic_Talys
inelasticBaghdadDataframe["Type"]="Inelastic_Baghdad"

In [64]:
#show the processed dataframe
inelasticBaghdadDataframe

,Isotope,Energy (keV),XS (mb),Compound,Element,Type
86,00Al,2211.80,68.141,27Al,Al,Inelastic_Baghdad
85,00Al,843.75,78.624,27AI,Al,Inelastic_Baghdad
84,00Al,1014.40,131.040,27Al,Al,Inelastic_Baghdad
219,00Ca,3904.20,10.296,40Ca,Ca,Inelastic_Baghdad
218,00Ca,3736.90,12.664,40Ca,Ca,Inelastic_Baghdad
217,00Ca,770.30,21.004,40K,Ca,Inelastic_Baghdad
557,00Fe,1810.50,32.292,56Fe,Fe,Inelastic_Baghdad
556,00Fe,1238.30,49.140,58Fe,Fe,Inelastic_Baghdad
555,00Fe,846.78,468.000,56Fe,Fe,Inelastic_Baghdad
60,00Mg,1808.60,14.414,26Mg,Mg,Inelastic_Baghdad


In [65]:
#set the address for the Natural_Radiation file
naturalRadiationFileAddress="C:/ML4Sci/NASA-gamma/nasagamma/data/Natural_radiation.csv"

In [66]:
#read the .csv file into a dataframe
naturalRadiationDataframe=pd.read_csv(naturalRadiationFileAddress)

In [67]:
#show the read dataframe
naturalRadiationDataframe

,Isotope,Energy (keV),Intensity (%),Info
0,40K,1460.820,10.660,Nat-40K
1,208Tl,722.040,0.201,Nat-232Th
2,208Tl,2614.533,99.000,Nat-232Th
3,208Tl,510.770,22.600,Nat-232Th
4,208Tl,860.564,12.420,Nat-232Th
...,...,...,...,...
133,235U,185.712,57.200,Nat-235U
134,235U,279.500,0.270,Nat-235U
135,235U,221.399,0.120,Nat-235U
136,235U,194.940,0.630,Nat-235U


In [68]:
#retain rows for only the elements of interest
naturalRadiationDataframe=naturalRadiationDataframe[naturalRadiationDataframe['Info'].str.endswith(tuple(elementsOfInterest))]

In [69]:
#remove any rows that are outside the relavent energy range
naturalRadiationDataframe=naturalRadiationDataframe[naturalRadiationDataframe["Energy (keV)"].between(relaventEnergyBands[0],relaventEnergyBands[-1])]

In [70]:
#create a temporary variable to be used to create the dataframe containing only the sampled lines
temp=[]
#iterate through the elements of interest
for currentElement in elementsOfInterest:
    #get the rows corresponding to the current element
    currentElementFrame=naturalRadiationDataframe[naturalRadiationDataframe['Info'].str.endswith(tuple(currentElement))]
    #sort the dataframe in increasing order of cross-section
    currentElementBFrame=currentElementFrame.sort_values('Intensity (%)')
    #remove any values less than a certain Sigma threshold
    currentElementFrame=currentElementFrame[currentElementFrame['Intensity (%)']>=0]
    #sample the last few rows (number specified in the beginning)
    currentElementFrame=currentElementFrame.iloc[-1*noOfLinesPerElementPerDataset:,:]
    #add a column with the name of the current element
    currentElementFrame['Element']=currentElement
    #save the extracted rows
    if currentElement=='Al':
        temp=currentElementFrame
    else:
        temp=pd.concat([temp,currentElementFrame])
#save temp as the Bagddad dataframe
naturalRadiationDataframe=temp

In [71]:
#add a column Type and set it to Inelastic_Talys
naturalRadiationDataframe["Type"]="Natural_radiation"

In [72]:
#replace isotope by info
naturalRadiationDataframe["Isotope"]=naturalRadiationDataframe["Info"]

In [73]:
#show the processed dataframe
naturalRadiationDataframe

,Isotope,Energy (keV),Intensity (%),Info,Element,Type
0,Nat-40K,1460.820,10.660,Nat-40K,K,Natural_radiation
114,Nat-232Th,840.377,0.910,Nat-232Th,Th,Natural_radiation
115,Nat-232Th,1495.930,0.860,Nat-232Th,Th,Natural_radiation
116,Nat-232Th,1459.138,0.830,Nat-232Th,Th,Natural_radiation
75,Nat-238U,1207.680,0.451,Nat-238U,U,Natural_radiation
125,Nat-238U,766.380,0.294,Nat-238U,U,Natural_radiation
127,Nat-238U,1001.300,0.837,Nat-238U,U,Natural_radiation


In [74]:
#show all three processed dataframes

In [75]:
#show the processed dataframe
captureIAEADataframe

,Isotope,Energy (keV),Sigma (mb),DSigma (mb),Element,Type
860,27Al,4259.534,15.300,0.300,Al,Capture_IAEA
859,27Al,3033.896,17.900,0.300,Al,Capture_IAEA
858,27Al,7724.027,49.300,1.500,Al,Capture_IAEA
2609,40Ca,4418.520,70.800,1.800,Ca,Capture_IAEA
2606,40Ca,6419.590,176.000,5.000,Ca,Capture_IAEA
2604,40Ca,1942.670,352.000,7.000,Ca,Capture_IAEA
5848,56Fe,6018.532,227.000,5.000,Fe,Capture_IAEA
5846,56Fe,7645.545,549.000,11.000,Fe,Capture_IAEA
5845,56Fe,7631.136,653.000,13.000,Fe,Capture_IAEA
577,24Mg,2828.172,24.000,0.800,Mg,Capture_IAEA


In [76]:
#show the processed dataframe
inelasticBaghdadDataframe

,Isotope,Energy (keV),XS (mb),Compound,Element,Type
86,00Al,2211.80,68.141,27Al,Al,Inelastic_Baghdad
85,00Al,843.75,78.624,27AI,Al,Inelastic_Baghdad
84,00Al,1014.40,131.040,27Al,Al,Inelastic_Baghdad
219,00Ca,3904.20,10.296,40Ca,Ca,Inelastic_Baghdad
218,00Ca,3736.90,12.664,40Ca,Ca,Inelastic_Baghdad
217,00Ca,770.30,21.004,40K,Ca,Inelastic_Baghdad
557,00Fe,1810.50,32.292,56Fe,Fe,Inelastic_Baghdad
556,00Fe,1238.30,49.140,58Fe,Fe,Inelastic_Baghdad
555,00Fe,846.78,468.000,56Fe,Fe,Inelastic_Baghdad
60,00Mg,1808.60,14.414,26Mg,Mg,Inelastic_Baghdad


In [77]:
#show the processed dataframe
naturalRadiationDataframe

,Isotope,Energy (keV),Intensity (%),Info,Element,Type
0,Nat-40K,1460.820,10.660,Nat-40K,K,Natural_radiation
114,Nat-232Th,840.377,0.910,Nat-232Th,Th,Natural_radiation
115,Nat-232Th,1495.930,0.860,Nat-232Th,Th,Natural_radiation
116,Nat-232Th,1459.138,0.830,Nat-232Th,Th,Natural_radiation
75,Nat-238U,1207.680,0.451,Nat-238U,U,Natural_radiation
125,Nat-238U,766.380,0.294,Nat-238U,U,Natural_radiation
127,Nat-238U,1001.300,0.837,Nat-238U,U,Natural_radiation


In [78]:
#concatenate the three dataframes together
allLinesDataset=pd.concat([captureIAEADataframe,
                           inelasticBaghdadDataframe,
                           naturalRadiationDataframe])

In [79]:
#show the concattenated dataset after sorting in increasing order of energy
allLinesDataset=allLinesDataset.sort_values(['Element',
                                             'Energy (keV)'
                                            ],
                                            ascending=[True,
                                                       True
                                                      ]
                                           )
allLinesDataset

,Isotope,Energy (keV),Sigma (mb),DSigma (mb),Element,Type,XS (mb),Compound,Intensity (%),Info
85,00Al,843.750,NaN,NaN,Al,Inelastic_Baghdad,78.624,27AI,NaN,NaN
84,00Al,1014.400,NaN,NaN,Al,Inelastic_Baghdad,131.040,27Al,NaN,NaN
86,00Al,2211.800,NaN,NaN,Al,Inelastic_Baghdad,68.141,27Al,NaN,NaN
859,27Al,3033.896,17.9,0.3,Al,Capture_IAEA,NaN,NaN,NaN,NaN
860,27Al,4259.534,15.3,0.3,Al,Capture_IAEA,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
125,Nat-238U,766.380,NaN,NaN,U,Natural_radiation,NaN,NaN,0.294,Nat-238U
127,Nat-238U,1001.300,NaN,NaN,U,Natural_radiation,NaN,NaN,0.837,Nat-238U
7287,00U,1015.100,NaN,NaN,U,Inelastic_Baghdad,124.862,238U,NaN,NaN
75,Nat-238U,1207.680,NaN,NaN,U,Natural_radiation,NaN,NaN,0.451,Nat-238U


In [81]:
#set the address where the sampled lines are to be saved
processedLinesDataset="C:/ML4Sci/Ml4Sci_GRS_abundance_estimation/linesSampledFromNASAGamma.csv"
#save the lines sampled from all elements as a csv
allLinesDataset.to_csv(processedLinesDataset,
                       index=False)

In [86]:
allLinesDataset[allLinesDataset['Isotope'].str.endswith(tuple('Si'))]

,Isotope,Energy (keV),Sigma (mb),DSigma (mb),Element,Type,XS (mb),Compound,Intensity (%),Info
358,00Ti,889.240,NaN,NaN,Si,Inelastic_Baghdad,34.955,46Ti,NaN,NaN
356,00Ti,983.490,NaN,NaN,Si,Inelastic_Baghdad,360.360,48Ti,NaN,NaN
4672,48Ti,1381.745,5180.0,120.0,Si,Capture_IAEA,NaN,NaN,NaN,NaN
109,00Si,1778.800,NaN,NaN,Si,Inelastic_Baghdad,126.360,28Si,NaN,NaN
4674,48Ti,6418.426,1960.0,60.0,Si,Capture_IAEA,NaN,NaN,NaN,NaN
4673,48Ti,6760.084,2970.0,90.0,Si,Capture_IAEA,NaN,NaN,NaN,NaN
358,00Ti,889.240,NaN,NaN,Ti,Inelastic_Baghdad,34.955,46Ti,NaN,NaN
356,00Ti,983.490,NaN,NaN,Ti,Inelastic_Baghdad,360.360,48Ti,NaN,NaN
4672,48Ti,1381.745,5180.0,120.0,Ti,Capture_IAEA,NaN,NaN,NaN,NaN
109,00Si,1778.800,NaN,NaN,Ti,Inelastic_Baghdad,126.360,28Si,NaN,NaN
